## Study the Dynamics of the many-body system

Imports

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
from src.qutip_lab.qutip_class import SpinOperator, SpinHamiltonian, SteadyStateSolver
import qutip
from typing import List
import os

class Driving:
    def __init__(
        self, h_i: np.array, h_f: np.array, rate: float, idx: int, direction: int
    ) -> None:
        self.hi = h_i
        self.hf = h_f
        self.rate = rate
        self.idx: int = idx
        self.direction = direction

    def field(self, t: float, args):
        return (
            self.hi[self.direction, self.idx] * np.exp(-t * self.rate)
            + (1 - np.exp(-t * self.rate)) * self.hf[self.direction, self.idx]
        )

    def get_the_field(self, t: np.ndarray):
        return (
            self.hi[None, self.direction, :] * np.exp(-t[:, None] * self.rate)
            + (1 - np.exp(-t[:, None] * self.rate)) * self.hf[None, self.direction, :]
        )


class PeriodicDriving:
    def __init__(
        self, h_i: np.array, delta: np.array, rate: float, idx: int, direction: int
    ) -> None:
        self.hi = h_i
        self.delta = delta
        self.rate = rate
        self.idx: int = idx
        self.direction = direction

    def field(self, t: float, args):
        return self.hi[self.direction, self.idx] + (
            self.delta[self.direction, self.idx]
        ) * np.sin(self.rate * t)

    def get_the_field(self, t: np.ndarray):
        return (
            self.hi[None, self.direction, :]
            + (self.delta[None, self.direction, :]) * np.sin(self.rate * t)[:, None]
        )



Data

In [5]:
l=8
rate=0.1
periodic=False

steps = 1000
tf = 10.0
time = np.linspace(0.0, tf, steps)

hi = np.ones((2, l))
hi[1] = 2.0  # high transverse field
hi[0] = 0.001
# define the final external field
hf = np.ones((2, l))
hf[1] = 1.0
hf[0] = 0.001


# define the delta for the periodic driving
delta = np.ones((2, l))
delta[1] = 0.9
delta[0] = 0.0

In [6]:
ham0 = SpinHamiltonian(
direction_couplings=[("z", "z")],
pbc=True,
coupling_values=[1.0],
size=l,
)

ham1 = SpinHamiltonian(
    direction_couplings=[("x", "x")],
    pbc=True,
    coupling_values=[1.0],
    size=l,
)

hamExtX = SpinOperator(
    index=[("x", i) for i in range(l)], coupling=hi[1], size=l
)
hamExtZ = SpinOperator(
    index=[("z", i) for i in range(l)], coupling=hi[0], size=l
)

eng, psi0 = np.linalg.eigh(ham0.qutip_op + hamExtZ.qutip_op + hamExtX.qutip_op)
psi0 = qutip.Qobj(psi0[:, 0], shape=psi0.shape, dims=([[2 for i in range(l)], [1]]))

print("real ground state energy=", eng[0])
# to check if we have the same outcome with the Crank-Nicholson algorithm
# psi = initialize_psi_from_z_and_x(z=-1 * zi[0], x=zi[1])
# psi = psi.detach().numpy()
# for i in range(l):
#     psi_l = qutip.Qobj(psi[i], shape=psi[i].shape, dims=([[2], [1]]))
#     if i == 0:
#         psi0 = psi_l
#     else:
#         psi0 = qutip.tensor(psi0, psi_l)
# compute and check the magnetizations
obs: List[qutip.Qobj] = []
obs_x: List[qutip.Qobj] = []
for i in range(l):
    z_op = SpinOperator(index=[("z", i)], coupling=[1.0], size=l, verbose=1)
    # print(f"x[{i}]=", x.qutip_op, "\n")
    x_op = SpinOperator(index=[("x", i)], coupling=[1.0], size=l, verbose=0)
    obs.append(z_op.qutip_op)
    obs_x.append(x_op.qutip_op)

print("\n INITIALIZE THE HAMILTONIAN \n")
# build up the time dependent object for the qutip evolution
hamiltonian = [ham0.qutip_op]

print("periodic=", periodic, "\n")
for i in range(l):
    if periodic:
        drive_z = PeriodicDriving(
            h_i=hi,
            delta=delta,
            rate=rate,
            idx=i,
            direction=0,
        )
    else:
        drive_z = Driving(
            h_i=hi,
            h_f=hf,
            rate=rate,
            idx=i,
            direction=0,
        )

    hamiltonian.append([obs[i], drive_z.field])

h_z = drive_z.get_the_field(time).reshape(time.shape[0], 1, -1)
for i in range(l):
    if periodic:
        drive_x = PeriodicDriving(
            h_i=hi,
            delta=delta,
            rate=rate,
            idx=i,
            direction=1,
        )
    else:
        drive_x = Driving(
            h_i=hi,
            h_f=hf,
            rate=rate,
            idx=i,
            direction=1,
        )
    hamiltonian.append([obs_x[i], drive_x.field])
h_x = drive_x.get_the_field(time).reshape(time.shape[0], 1, -1)

h = np.append(h_z, h_x, axis=1)
print(h.shape)

# evolution

output = qutip.sesolve(hamiltonian, psi0, time, e_ops=obs + obs_x)

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

State dynamics

In [ ]:
state=output.states
print(state.shape)